In [1]:
from typing import List
import numpy as np
import geopandas as gpd

import os
import pandas as pd
from shapely.geometry import shape  


In [5]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.parent.parent))
from demeter.tea.src.geometry_utils import get_wgs84_gdf_for_aoi

ei_dir = '/Users/alicedurieux/code/earth-index-ml/'


In [4]:
%load_ext autoreload
%autoreload 2


# Get df of MGRS tiles intersecting RA AOI

In [5]:
aoi_file = f'{ei_dir}demeter/geometry/aois/ra_aoi_indonesia.geojson'
df = get_wgs84_gdf_for_aoi(aoi_file, f'{ei_dir}demeter/geometry/wgs84_tiles/wgs_tiles_intersecting_indonesia_aoi.geojson')


# Get small tiles intersecting points

In [ ]:
points = gpd.read_file(f'{ei_dir}demeter/geometry/tea_geometries_test_palm_samples.geojson')
points.head()


In [ ]:
points_with_tile_id = gpd.sjoin(points, df, how="left", predicate="intersects")

# Optionally, you can keep only the relevant columns
points_with_tile_id = points_with_tile_id[['probability', 'geometry', 'mgrs_id']]

#Add unique ID for points
points_with_tile_id['unique_point_id'] = points_with_tile_id.index


In [10]:
# iterate over mgrs_ids and get tiles intersecting points

intersecting_geometries = []

# Group by mgrs_id
for mgrs_id, group in points_with_tile_id.groupby('mgrs_id'):
        # Construct the directory path
    parquet_directory = f'{ei_dir}demeter/geometry/wgs84_tiles/tiles_intersecting_indonesia_aoi/'
    # Find parquet files where the first part of the name matches the mgrs_id
    parquet_file = [f for f in os.listdir(parquet_directory) if f.split('_')[0] == mgrs_id]
    # Read the parquet file
    tile_gdf = gpd.read_parquet(parquet_directory + parquet_file[0])
    tile_gdf.to_crs('epsg:4326', inplace=True)
    tile_gdf['tile_geometry'] = tile_gdf.geometry

 # Perform a spatial join to keep all points and add tile geometry if contained
    joined = gpd.sjoin(group, tile_gdf, how='left', predicate='within')
        
        # Add the results to the list
    intersecting_geometries.append(joined)
# Concatenate all intersecting geometries into a single GeoDataFrame
result_df = gpd.GeoDataFrame(pd.concat(intersecting_geometries, ignore_index=True))

# save points with tile id
result_df.drop(columns=['index_right'], inplace=True)
result_df.drop('tile_geometry', axis=1).to_file('points_with_tile_id_intersecting_indonesia_aoi.geojson', driver='GeoJSON')

# save tiles intersecting points 
tiles = result_df.drop('geometry', axis=1).rename(columns={'tile_geometry': 'geometry'}).drop_duplicates(subset=['tile_id'])
tiles.to_file('tiles_intersecting_points.geojson', driver='GeoJSON')
